# COMP7095 - Big Data Management

## Spark Lab 3: Spark SQL

### Introduction
Spark SQL is a Spark module for structured data processing. It provides a programming abstraction called DataFrames, which can also act as a distributed SQL query engine. In this lab, we learn how to manipulate the data using the functions provided by the dataframe and SQL queries.

### Preparation
It is assumed that you have installed Python 3.9.x and created a virtual environment on your computer. Next, we need to perform the following steps for this lab:

1. Download the `ipnb version` of this lab and `movie_reviews.tsv` and save them.
2. Launch Terminal/Command prompt.
3. Start your Spark with Jupyter Notebook:

   


### Creating DataFrame from RDD

Everything now is ready, we can go ahead to work on this lab. 

First, we import the required packages.

In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *

We define a function named "preprocess" that will be used to split the values (review and sentiment) of each line.

In [2]:
def preprocess(line):
    values = line.split('\t')
    return values[1], values[0]

Then, we get the instance of the Spark context and load the data file to create a resilient distributed data (RDD) object. 

We use the `filter` function to ignore the header row and pass the data to the `preprocess` function. Then, a new RDD object will be created.

In [3]:
sc = pyspark.SparkContext.getOrCreate()

rdd = sc.textFile('/Users/Chase/movie_reviews.tsv')
reviews = rdd.filter(lambda x: x != 'review\tsentiment').map(preprocess)

We can check the content of the RDD object using the `take` function.

In [4]:
reviews.take(1)

[('positive',
  "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word. It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away. I would say the main appeal of the show is due to the fact that it goes w

With SQLContext, we can create a dataframe from a RDD object. \
<mark>DataFrame = RDD + Schema</mark>

To define a schema, we need the `StructField` function to describe each column. The syntax of the `StructField` function is: \

`StructField(col_name, col_type, nullable)`

In [5]:
schema = StructType([
    StructField('review', StringType()),
    StructField('sentiment', StringType())
])

# sqlContext = SQLContext(sc)
# df = sqlContext.createDataFrame(reviews, schema)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(reviews, schema)

You review the schema attribute of the dataframe.

In [6]:
df.schema

StructType([StructField('review', StringType(), True), StructField('sentiment', StringType(), True)])

To view the content of the dataframe, we use the `show` function.

In [7]:
df.toPandas()

,review,sentiment
0,positive,One of the other reviewers has mentioned that ...
1,positive,A wonderful little production. The filming tec...
2,positive,I thought this was a wonderful way to spend ti...
3,negative,Basically there's a family where a little boy ...
4,positive,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...
49995,positive,I thought this movie did a down right good job...
49996,negative,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,negative,I am a Catholic taught in parochial elementary...
49998,negative,I'm going to have to disagree with the previou...


### Creating DataFrame from Data File

Import the required packages and create a SQL context.

In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

Create a schema and use the SQL context to load the data from the file - `movie_reviews.tsv`.

In [9]:
schema = StructType([
    StructField('review', StringType()),
    StructField('sentiment', StringType())
])

df2 = spark.read.csv('/Users/Chase/movie_reviews.tsv', header=True, schema=schema, sep='\t')
df2.toPandas()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


User Define Function (UDF) allows us to create new columns based on the existing columns. 
For example, we want new columns to: \
- present the length of the review
- use Boolean (True/False) to present the sentiment of the review
- present how many "funny" included in the review
- present how many "terrible" include in the review

In [10]:
from pyspark.sql.functions import udf

length = udf(lambda x: len(x))
pos = udf(lambda x: x == 'positive')
funny = udf(lambda r, s: r.count('funny') if s == 'positive' else 0)
terrible = udf(lambda r, s: r.count('terrible') if s == 'negative' else 0)

Use the UDFs to create new columns.

In [11]:
df2 = df2.withColumn('length', length('review'))
df2 = df2.withColumn('positive', pos('sentiment'))
df2 = df2.withColumn('funny', funny('review', 'sentiment'))
df2 = df2.withColumn('negative', pos('sentiment'))
df2 = df2.withColumn('terrible', terrible('review', 'sentiment'))

We can also delete the unwanted column by using the `drop` function.

In [12]:
df2 = df2.drop('sentiment')

Let's see what is the result!

In [13]:
df2.toPandas()

,review,length,positive,funny,negative,terrible
0,One of the other reviewers has mentioned that ...,1728,true,0,true,0
1,A wonderful little production. The filming tec...,962,true,0,true,0
2,I thought this was a wonderful way to spend ti...,904,true,0,true,0
3,Basically there's a family where a little boy ...,715,false,0,false,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1272,true,0,true,0
...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,986,true,0,true,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",620,false,0,false,0
49997,I am a Catholic taught in parochial elementary...,1258,false,0,false,0
49998,I'm going to have to disagree with the previou...,1233,false,0,false,0


### Caching
Spark provides an important feature to cache intermediate data and provide significant performance improvement while running multiple queries on the same data.

By default, the dataframe is not cached. We can check its status through the `is_cached` attribute.

In [14]:
df2.is_cached

False

To cache the data, we simply call the cache function of the dataframe.

In [15]:
df2.cache()

DataFrame[review: string, length: string, positive: string, funny: string, negative: string, terrible: string]

We can verify it by checking the `is_cached` attribute again.

In [16]:
df2.is_cached

True

We can remove the cache by using the `unpersist` function.

`df2.unpersist()`

Of course, we want to keep using the caching for the following parts.

### Data Exploring

DataFrame provides different functions for retrieving data.

#### Ordering
We change the display order using the `orderBy` function. For example, sort by the "positive" column in ascending order.

Note that `ascending=True` means sort in ascending order; and `ascending=False` means sort in decending order.

In [17]:
df2.orderBy('positive', ascending=True).toPandas()

,review,length,positive,funny,negative,terrible
0,Basically there's a family where a little boy ...,715,false,0,false,0
1,"This show was an amazing, fresh & innovative i...",923,false,0,false,0
2,Encouraged by the positive comments about this...,669,false,0,false,0
3,Phil the Alien is one of those quirky films wh...,534,false,0,false,0
4,I saw this movie when I was about 12 when it c...,915,false,0,false,0
...,...,...,...,...,...,...
49995,"I loved it, having been a fan of the original ...",695,true,0,true,0
49996,Imaginary Heroes is clearly the best film of t...,1144,true,0,true,0
49997,I got this one a few weeks ago and love it! It...,967,true,0,true,0
49998,John Garfield plays a Marine who is blinded by...,968,true,0,true,0


#### Your task 1: Please sort df2 according to 'negative' in descending order.

In [18]:
df2.orderBy('negative', ascending=False).toPandas()

,review,length,positive,funny,negative,terrible
0,One of the other reviewers has mentioned that ...,1728,true,0,true,0
1,A wonderful little production. The filming tec...,962,true,0,true,0
2,I thought this was a wonderful way to spend ti...,904,true,0,true,0
3,"Petter Mattei's ""Love in the Time of Money"" is...",1272,true,0,true,0
4,"Probably my all-time favorite movie, a story o...",656,true,0,true,0
...,...,...,...,...,...,...
49995,This is your typical junk comedy. There are al...,701,false,0,false,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",620,false,0,false,0
49997,I am a Catholic taught in parochial elementary...,1258,false,0,false,0
49998,I'm going to have to disagree with the previou...,1233,false,0,false,0


#### Ordering by Multiple Columns
We can sort the data by multiple columns too.

In [19]:
df2.orderBy(['positive','terrible'], ascending=[True, False]).toPandas()

,review,length,positive,funny,negative,terrible
0,Less than 10 minutes into this film I wanted i...,3318,false,0,false,7
1,OK so I hear about this new Justin Timberlake ...,2424,false,0,false,6
2,"Hey guys and girls! Don't ever rent, or may Go...",1058,false,0,false,6
3,Oh. Good. Grief. I saw this movie title in the...,1263,false,0,false,5
4,"*** Warning - this review contains ""plot spoil...",7673,false,0,false,5
...,...,...,...,...,...,...
49995,"I loved it, having been a fan of the original ...",695,true,0,true,0
49996,Imaginary Heroes is clearly the best film of t...,1144,true,0,true,0
49997,I got this one a few weeks ago and love it! It...,967,true,0,true,0
49998,John Garfield plays a Marine who is blinded by...,968,true,0,true,0


#### Selecting Columns
The `select` function allows us to select which columns we want to display. For example, we want to have "review", "positive", and "terrible" columns only.

In [20]:
df2.select(['review', 'positive', 'terrible']).toPandas()

,review,positive,terrible
0,One of the other reviewers has mentioned that ...,true,0
1,A wonderful little production. The filming tec...,true,0
2,I thought this was a wonderful way to spend ti...,true,0
3,Basically there's a family where a little boy ...,false,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",true,0
...,...,...,...
49995,I thought this movie did a down right good job...,true,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",false,0
49997,I am a Catholic taught in parochial elementary...,false,0
49998,I'm going to have to disagree with the previou...,false,0


#### Your task 2: Please list df2 by 'review', 'length' and 'funny'.

In [21]:
df2.select(['review', 'length', 'funny']).toPandas()

,review,length,funny
0,One of the other reviewers has mentioned that ...,1728,0
1,A wonderful little production. The filming tec...,962,0
2,I thought this was a wonderful way to spend ti...,904,0
3,Basically there's a family where a little boy ...,715,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1272,0
...,...,...,...
49995,I thought this movie did a down right good job...,986,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",620,0
49997,I am a Catholic taught in parochial elementary...,1258,0
49998,I'm going to have to disagree with the previou...,1233,0


#### Adding Conditions
With the `where` function, we can specify the condition for data retrieval. For example, we want the negative reviews with more than 3000 characters.

In [22]:
df2.select('review', 'positive', 'length').where('positive = false and length > 3000').toPandas()

,review,positive,length
0,"Maybe it was the title, or the trailer (certai...",false,3353
1,"Okay, last night, August 18th, 2004, I had the...",false,4140
2,From the film's first shot - Keira Knightley a...,false,5188
3,"Though I'd heard that ""Cama de Gato"" was the w...",false,4685
4,The day has finally come for me to witness the...,false,3091
...,...,...,...
1491,"""Air Bud 2: Golden Receiver"" is a very bad reh...",false,3708
1492,"What a disaster! Normally, when one critiques ...",false,4124
1493,"It is the early morning of our discontent, and...",false,5767
1494,"My thoughts on the movie, 9 It was not good, n...",false,3358


#### Aggregate
With the `agg` (aggregate) functions, we can find the `min`, `max`, `avg`, `stddev`, and `count` from the dataframe.

For example, we want to find the maximum number of "funny" words in a single review.

In [23]:
df2.agg({'funny':'max'}).show()

+----------+
|max(funny)|
+----------+
|         9|
+----------+



#### Your task 3: Please find the maximum number of "terrible" words in a single review.

In [24]:
df2.agg({'terrible':'max'}).show()

+-------------+
|max(terrible)|
+-------------+
|            7|
+-------------+



#### Grouping
Combining with `groupBy` function, we can find the aggregates of different groups. 

For example, we want to find the average lengths of positive reviews and negative reviews respectively.

In [25]:
df2.groupBy('positive').agg({'length':'avg'}).toPandas()

,positive,avg(length)
0,false,1270.822116
1,true,1302.940348


#### Summary
We can also show the simple statistic by using the `summary` function.

In [26]:
df2.summary().toPandas()

,summary,review,length,positive,funny,negative,terrible
0,count,50000,50000,50000,50000,50000,50000
1,mean,None,1286.87802,None,0.08072,None,0.05394
2,stddev,None,972.3387667034699,None,0.3625009099825748,None,0.28027138177300376
3,min,A Turkish Bath sequence in a film noir loc...,1000,false,0,false,0
4,25%,None,690.0,None,0.0,None,0.0
5,50%,None,954.0,None,0.0,None,0.0
6,75%,None,1560.0,None,0.0,None,0.0
7,max,ý thýnk uzak ýs the one of the best films of a...,999,true,9,true,7


### Using SQL Query
We can create a temp view from the dataframe and the view can be used with SQL queries to retrieve the data.

In [27]:
df2.createOrReplaceTempView('movie_reviews')

Now, let's have a try to get something with a simple SQL query. For example, we want to show the columns including `review` and `length`. So, we use:

In [28]:
spark.sql('select review, length from movie_reviews').toPandas()

,review,length
0,One of the other reviewers has mentioned that ...,1728
1,A wonderful little production. The filming tec...,962
2,I thought this was a wonderful way to spend ti...,904
3,Basically there's a family where a little boy ...,715
4,"Petter Mattei's ""Love in the Time of Money"" is...",1272
...,...,...
49995,I thought this movie did a down right good job...,986
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",620
49997,I am a Catholic taught in parochial elementary...,1258
49998,I'm going to have to disagree with the previou...,1233


### Your task 4
Please show the reviews that contains at least 6 "funny" words or at least 6 "terrible" words. Also, we need them reviews are sorted by the number of the "funny" words in decending order and then the number of the "terrible" words in decending order.

In [32]:
spark.sql('select review, funny, terrible from movie_reviews').where('funny >= 6 or terrible >= 6').orderBy(['funny','terrible'], ascending=[False, False]).toPandas()

,review,funny,terrible
0,"I grew up with the Abbott and Costello movies,...",9,0
1,"During my trip in a youth leadership forum, I ...",7,0
2,I've watched this movie on a fairly regular ba...,7,0
3,8 Simple Rules for Dating My Teenage Daughter ...,6,0
4,Fantastic Chaplin movie with many memorable mo...,6,0
5,"This movie is just funny. mindless, but funny....",6,0
6,There is a lot wrong with this film. I will no...,6,0
7,Less than 10 minutes into this film I wanted i...,0,7
8,OK so I hear about this new Justin Timberlake ...,0,6
9,"Hey guys and girls! Don't ever rent, or may Go...",0,6


We can also get the average lengths of the positive and negative reviews respectively.

In [33]:
spark.sql('select positive, avg(length) as avg_len from movie_reviews group by positive').toPandas()

,positive,avg_len
0,false,1270.822116
1,true,1302.940348


### After using Spark
In the end, we should stop the Spark by using the `stop` function.

In [34]:
sc.stop()
spark.stop()